In [1]:
import transformers, datasets

In [1]:
! nvidia-smi

Sat Aug 27 20:24:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   47C    P0   143W / 250W |  11248MiB / 12198MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:DB:00.0 Off |                    0 |
| N/A   

In [4]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /home/somnath-am/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
! nvidia-smi

Tue Aug 23 23:20:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   61C    P0   178W / 250W |  10876MiB / 12198MiB |     95%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:DB:00.0 Off |                    0 |
| N/A   

In [5]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(datasets["train"])

,text
0,"On September 12 , satellite images indicated that the eighteenth tropical depression of the season formed southwest of Cabo San Lucas . Located in close proximity to Tropical Storm Norbert , the depression would soon enter a region of colder sea surface temperatures , and the National Hurricane Center noted that only very little intensification was possible . As predicted , Norbert hindered further intensification of Tropical Depression Eighteen @-@ E. The National Hurricane Center noted six hours later that the depression would likely be absorbed into Norbert . The National Hurricane Center later believed that Tropical Depression Eighteen @-@ E was "" dominating "" the interaction between the two systems , and was forecast to absorb Norbert . Unlike the latter prediction , Norbert absorbed Tropical Depression Eighteen @-@ E later that day . \n"
1,
2,
3,= = = Architectural style = = = \n
4,
5,"Kajal Aggarwal as Mithravinda Devi and Indu . Mithravinda Devi is the crown princess of Udayghad who is in love with Bhairava and also dies in 1609 . In 2009 , her reincarnation is Indira ( alias Indu ) , a carefree student . She falls in love with Harsha but believes him to be her father 's murderer because of Raghuveer 's deception . She reunites with Harsha in the end after remembering her past life . \n"
6,"ზ , ო , and ხ ( zeni , oni , khani ) are almost always written without the small tick at the end , while the handwritten form of ჯ ( jani ) often uses a vertical line , ( sometimes with a taller ascender , or with a diagonal cross bar ) ; even when it is written at a diagonal , the cross @-@ bar is generally shorter than in print . \n"
7,"The action was the first of two in this campaign in which lone British frigates were briefly overwhelmed by superior French forces as they sailed independently to join Rowley 's squadron . On each occasion however , Rowley was able to recapture the lost frigate and drive off the French attackers . Corbet 's action was particularly violent , British casualties totalling 49 killed and 114 wounded , including every single officer and all but three of the soldiers embarked . Africaine was seriously damaged and would not be ready to return to active service for some months . French losses were less severe , Astrée suffering one killed and two wounded , Iphigénie nine killed and 33 wounded . \n"
8,"Upon returning to the freighter , Keamy unsuccessfully attempts to kill Michael Dawson ( Harold Perrineau ) , whom he has discovered is Ben 's spy , then obtains the "" secondary protocol "" from a safe . The protocol contains instructions from Widmore for finding Ben if he finds out Keamy 's intention to torch the island , which he apparently had . The protocol contains details about a 1980s research station called the "" Orchid "" that was previously run by a group of scientists working for the Dharma Initiative . Keamy is also informed by Captain Gault that Keamy and his mercenary squad may be suffering from some sort of mental sickness , a notion Keamy dismisses . Later in the day , Omar straps a dead man 's switch to Keamy , rigged to detonate C4 on the freighter if Keamy 's heart stops beating . That night , Frank refuses to fly the mercenaries to the island . In a display of power , Keamy slits the throat of the ship 's doctor Ray ( Marc Vann ) and throws him overboard and later outdraws and shoots Captain Gault ( Grant Bowler ) during a tense standoff . Frank flies the remaining five mercenaries back to the island . On December 30 , the team apprehends Ben at the Orchid and takes him to the chopper where they are ambushed and killed by Ben 's people — referred to as the "" Others "" by the 815 survivors — and 815 survivors Kate Austen ( Evangeline Lilly ) and Sayid Jarrah ( Naveen Andrews ) . After a chase to recapture Ben and a brawl with Sayid , Keamy is shot in the back by Richard Alpert ( Nestor Carbonell ) , who leaves him for dead , unaware of Keamy 's bulletproof vest . Later , Keamy 

In [8]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"

In [9]:
from transformers import AutoTokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [12]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
block_size = 128
tokenized_datasets["train"][1]

In [15]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
from transformers import AutoConfig, AutoModelForMaskedLM

In [18]:
config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
No module named 'torch._C'